In [8]:
from inference_sdk import InferenceHTTPClient
import cv2

# ⚠️ reemplazá esto con tu API Key (la que ves en la página)
API_KEY = "SfEb7vZiIzSWMUlRWTWM"  # ejemplo: "SFeB7vZliZSMWUliRWTWM"
MODEL_ID = "soccer-whdhk/1"

# 1. Inicializar el cliente
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# 2. Imagen de prueba (podés cambiarla)
image_path = "image/prueba2.jpg"

# 3. Inferir
result = client.infer(image_path, model_id=MODEL_ID)
print(result)

# 4. Mostrar resultado visual
img = cv2.imread(image_path)
for pred in result["predictions"]:
    x, y, w, h = pred["x"], pred["y"], pred["width"], pred["height"]
    conf = pred["confidence"]
    label = pred["class"]

    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
    cv2.putText(img, f"{label} {conf:.2f}", (x1, max(12,y1-6)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

cv2.imshow("Predicciones", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{'inference_id': 'c7e43566-0ba0-437b-ae83-9e198b18a9e2', 'time': 0.04896724400009589, 'image': {'width': 1335, 'height': 808}, 'predictions': [{'x': 620.0, 'y': 636.5, 'width': 50.0, 'height': 121.0, 'confidence': 0.9080299139022827, 'class': 'player', 'class_id': 1, 'detection_id': '1ad1ab42-7508-4147-94e9-c1beb25225a7'}, {'x': 281.0, 'y': 502.5, 'width': 44.0, 'height': 121.0, 'confidence': 0.8931119441986084, 'class': 'player', 'class_id': 1, 'detection_id': '0937d1e8-78c1-470b-a30d-68660448cbb8'}, {'x': 632.0, 'y': 402.5, 'width': 44.0, 'height': 103.0, 'confidence': 0.8876462578773499, 'class': 'player', 'class_id': 1, 'detection_id': '2634e6bf-e216-4567-b3d5-9cf250f7c7fe'}, {'x': 289.0, 'y': 325.0, 'width': 32.0, 'height': 96.0, 'confidence': 0.8595064878463745, 'class': 'player', 'class_id': 1, 'detection_id': '0b85178f-e4c8-400a-bc1e-ae340db3e261'}, {'x': 641.0, 'y': 268.0, 'width': 40.0, 'height': 84.0, 'confidence': 0.8373066186904907, 'class': 'player', 'class_id': 1, 'detec

In [6]:
from inference_sdk import InferenceHTTPClient
import cv2
import numpy as np
import os

# === Configuración ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"
MODEL_ID = "soccer-whdhk/1"
IMG_PATH = "image/prueba1.jpg"  # imagen original

# === 1) Inicializar cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === 2) Cargar imagen ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"
draw = orig.copy()
H, W = orig.shape[:2]

# === 3) Inferencia ===
res = client.infer(IMG_PATH, model_id=MODEL_ID)
preds = res.get("predictions", [])

# === 4) Filtros ===
TH_PLAYER, TH_GOALIE, TH_REF, TH_BALL = 0.55, 0.55, 0.55, 0.30

def class_threshold(label):
    if label == "ball":     return TH_BALL
    if label == "goalie":   return TH_GOALIE
    if label == "referee":  return TH_REF
    return TH_PLAYER

# ROI (área verde)
hsv = cv2.cvtColor(orig, cv2.COLOR_BGR2HSV)
green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def mostly_on_pitch(x1, y1, x2, y2, min_ratio=0.35):
    x1, y1, x2, y2 = max(0,x1), max(0,y1), min(W-1,x2), min(H-1,y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return False
    return (crop > 0).mean() >= min_ratio

def plausible_person_box(x1, y1, x2, y2):
    w, h = max(1, x2 - x1), max(1, y2 - y1)
    ar = h / w
    return 1.1 <= ar <= 5.0

# === 5) Colores por clase ===
colors = {
    "player":  (0,255,0),
    "goalie":  (255,0,255),
    "referee": (0,165,255),
    "ball":    (0,255,255),
}

# === 6) Detecciones filtradas ===
kept = []
for p in preds:
    cls, conf = p["class"], float(p["confidence"])
    if conf < class_threshold(cls):
        continue

    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2): 
            continue
        if not mostly_on_pitch(x1, y1, x2, y2):
            continue

    kept.append((x1, y1, x2, y2, cls, conf))

# === 7) Dibujar detecciones ===
for (x1, y1, x2, y2, cls, conf) in kept:
    color = colors.get(cls, (0,255,0))
    cv2.rectangle(draw, (x1,y1), (x2,y2), color, 2)
    txt = f"{cls} {conf:.2f}"
    (tw, th), _ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(draw, (x1, max(0, y1- th - 6)), (x1+tw+6, y1), color, -1)
    cv2.putText(draw, txt, (x1+3, y1-3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)

# === 8) Mostrar y guardar ===
cv2.imshow("Original", orig)
cv2.imshow("Detecciones", draw)
side = cv2.hconcat([orig, draw])

# Crear carpeta de salida
os.makedirs("outputs", exist_ok=True)

# Generar nombre dinámico (según la imagen original)
base_name = os.path.splitext(os.path.basename(IMG_PATH))[0]

out_name_side = f"outputs/{base_name}_side_by_side.jpg"

cv2.imwrite(out_name_side, side)

cv2.waitKey(0)
cv2.destroyAllWindows()
print(f"Guardado: {out_name_side}")

Guardado: outputs/prueba1_side_by_side.jpg


In [15]:
from inference_sdk import InferenceHTTPClient
import cv2
import numpy as np
import os
import os.path as osp

# === Config ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"
MODEL_ID = "soccer-whdhk/1"
IMG_PATH = "image/prueba5.jpg"   # <-- poné acá tu archivo

# === Cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === Carga imagen y upscale suave ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"

H, W = orig.shape[:2]
target_long = 1280  # subir tamaño ayuda a ver objetos chicos
scale = max(1.0, target_long / max(H, W))
newW, newH = int(W * scale), int(H * scale)
img = cv2.resize(orig, (newW, newH), interpolation=cv2.INTER_CUBIC)
draw = img.copy()

# === Inferencia (con menor umbral base) ===
#   Tip: podés pasar el numpy array directamente
res = client.infer(img, model_id=MODEL_ID)  
preds = res.get("predictions", [])

# === Filtros mucho más suaves (o desactivables) ===
USE_PITCH_SOFT = True  # Soft en vez de hard
USE_AR_FILTER  = False # Apagado por ahora

# ROI "soft": si casi nada es césped, bajamos un poco la confianza; no eliminamos de una
if USE_PITCH_SOFT:
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def green_ratio(x1, y1, x2, y2):
    if not USE_PITCH_SOFT: return 1.0
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(newW-1, x2); y2 = min(newH-1, y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return 0.0
    return (crop > 0).mean()

def plausible_person_box(x1, y1, x2, y2):
    if not USE_AR_FILTER: return True
    w = max(1, x2 - x1); h = max(1, y2 - y1)
    ar = h / w
    return 0.9 <= ar <= 7.0

# Umbrales por clase (bajitos por dominio diferente)
TH = {"player":0.35, "goalie":0.35, "referee":0.35, "ball":0.25}

colors = {"player":(0,255,0), "goalie":(255,0,255), "referee":(0,165,255), "ball":(0,255,255)}

kept = []
for p in preds:
    cls  = p["class"]
    conf = float(p["confidence"])
    if conf < TH.get(cls, 0.35): 
        continue

    # coords del API son centro (x,y) y ancho/alto
    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2):
            continue
        gr = green_ratio(x1, y1, x2, y2)
        # si casi no hay césped debajo, penalizá confianza; no lo elimines
        if gr < 0.10:
            conf *= 0.6
            if conf < TH.get(cls, 0.35): 
                continue

    kept.append((x1, y1, x2, y2, cls, conf))

# Dibujar
for (x1, y1, x2, y2, cls, conf) in kept:
    color = colors.get(cls,(0,255,0))
    cv2.rectangle(draw, (x1,y1), (x2,y2), color, 2)
    label = f"{cls} {conf:.2f}"
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(draw, (x1, max(0, y1-th-6)), (x1+tw+6, y1), color, -1)
    cv2.putText(draw, label, (x1+3, y1-3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)

# Mostrar y guardar con nombre dinámico
side = cv2.hconcat([img, draw])

os.makedirs("outputs", exist_ok=True)
base = osp.splitext(osp.basename(IMG_PATH))[0]


out2 = f"outputs/{base}_side.jpg"


cv2.imwrite(out2, side)

cv2.imshow("Original (resize)", img)
cv2.imshow("Detecciones", draw)
cv2.waitKey(0); cv2.destroyAllWindows()

print("Guardado", out2)

Guardado outputs/prueba5_side.jpg


In [71]:
## MODELO 1

from inference_sdk import InferenceHTTPClient
import cv2
import numpy as np
import os
import os.path as osp

# === Config ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"
MODEL_ID = "soccer-whdhk/1"
IMG_PATH = "image/prueba10.jpg"   # <-- tu archivo

# === Cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === Carga imagen y upscale suave ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"

H, W = orig.shape[:2]
target_long = 1280  # subir tamaño ayuda a ver objetos chicos
scale = max(1.0, target_long / max(H, W))
newW, newH = int(W * scale), int(H * scale)
img = cv2.resize(orig, (newW, newH), interpolation=cv2.INTER_CUBIC)
draw = img.copy()

# === Inferencia (umbral base bajo) ===
res = client.infer(img, model_id=MODEL_ID)
preds = res.get("predictions", [])

# === Filtros suaves ===
USE_PITCH_SOFT = True
USE_AR_FILTER  = False

if USE_PITCH_SOFT:
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def green_ratio(x1, y1, x2, y2):
    if not USE_PITCH_SOFT: return 1.0
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(newW-1, x2); y2 = min(newH-1, y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return 0.0
    return (crop > 0).mean()

def plausible_person_box(x1, y1, x2, y2):
    if not USE_AR_FILTER: return True
    w = max(1, x2 - x1); h = max(1, y2 - y1)
    ar = h / w
    return 0.9 <= ar <= 7.0

TH = {"player":0.35, "goalie":0.35, "referee":0.35, "ball":0.25}
colors = {"player":(0,255,0), "goalie":(255,0,255), "referee":(0,165,255), "ball":(0,255,255)}

kept = []
for p in preds:
    cls  = p["class"]
    conf = float(p["confidence"])
    if conf < TH.get(cls, 0.35): 
        continue

    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2):
            continue
        gr = green_ratio(x1, y1, x2, y2)
        if gr < 0.10:
            conf *= 0.6
            if conf < TH.get(cls, 0.35): 
                continue

    kept.append((x1, y1, x2, y2, cls, conf))

# === Dibujar con etiquetas más pequeñas y finas ===
FONT_SCALE = 0.35          # antes 0.6
FONT_THICKNESS = 1        # antes 2
RECT_THICKNESS = 1        # rectángulo más fino
SHOW_CONF = True          # ponelo en False si querés ocultar el porcentaje

for (x1, y1, x2, y2, cls, conf) in kept:
    color = colors.get(cls, (0,255,0))
    label = f"{cls} {conf*100:.0f}%" if SHOW_CONF else cls

    # Rectángulo más fino
    cv2.rectangle(draw, (x1, y1), (x2, y2), color, RECT_THICKNESS)

    # Texto más chico, fondo proporcional
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, FONT_THICKNESS)
    cv2.rectangle(draw, (x1, max(0, y1 - th - 4)), (x1 + tw + 4, y1), color, -1)
    cv2.putText(draw, label, (x1 + 2, y1 - 3),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0,0,0),
                FONT_THICKNESS, cv2.LINE_AA)

# === Mostrar y guardar ===
side = cv2.hconcat([img, draw])
os.makedirs("outputs", exist_ok=True)
base = osp.splitext(osp.basename(IMG_PATH))[0]
out2 = f"outputs/{base}_side_by_side.jpg"

cv2.imwrite(out2, side)

cv2.imshow("Detecciones", draw)

# Espera hasta 10 segundos (10000 ms) o hasta que se presione una tecla
key = cv2.waitKey(10000)  # espera 10 segundos

# Si no se presiona nada (key == -1), cierra igual
cv2.destroyAllWindows()

print("Guardado:", out2)

Guardado: outputs/prueba10_side_by_side.jpg


In [72]:
## MODELO 2


from inference_sdk import InferenceHTTPClient
import cv2
import os, os.path as osp

# === Configuración ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"  # <--- poné tu clave Roboflow acá
MODEL_ID = "football-3oxbt/3"  # proyecto/version
IMG_PATH = "image/prueba10.jpg"  # tu imagen de prueba



# === Cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === Carga imagen y upscale suave ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"

H, W = orig.shape[:2]
target_long = 1280  # subir tamaño ayuda a ver objetos chicos
scale = max(1.0, target_long / max(H, W))
newW, newH = int(W * scale), int(H * scale)
img = cv2.resize(orig, (newW, newH), interpolation=cv2.INTER_CUBIC)
draw = img.copy()


# === Inferencia ===
res = client.infer(img, model_id=MODEL_ID)  # umbral 0.30
preds = res.get("predictions", [])

# === Filtros suaves ===
USE_PITCH_SOFT = True
USE_AR_FILTER  = False

if USE_PITCH_SOFT:
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def green_ratio(x1, y1, x2, y2):
    if not USE_PITCH_SOFT: return 1.0
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(newW-1, x2); y2 = min(newH-1, y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return 0.0
    return (crop > 0).mean()

def plausible_person_box(x1, y1, x2, y2):
    if not USE_AR_FILTER: return True
    w = max(1, x2 - x1); h = max(1, y2 - y1)
    ar = h / w
    return 0.9 <= ar <= 7.0

TH = {"player":0.35, "goalie":0.35, "referee":0.35, "ball":0.25}
colors = {"player":(0,255,0), "goalie":(255,0,255), "referee":(0,165,255), "ball":(0,255,255)}

kept = []
for p in preds:
    cls  = p["class"]
    conf = float(p["confidence"])
    if conf < TH.get(cls, 0.35): 
        continue

    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2):
            continue
        gr = green_ratio(x1, y1, x2, y2)
        if gr < 0.10:
            conf *= 0.6
            if conf < TH.get(cls, 0.35): 
                continue

    kept.append((x1, y1, x2, y2, cls, conf))

# === Dibujar con etiquetas más pequeñas y finas ===
FONT_SCALE = 0.35          # antes 0.6
FONT_THICKNESS = 1        # antes 2
RECT_THICKNESS = 1        # rectángulo más fino
SHOW_CONF = True          # ponelo en False si querés ocultar el porcentaje

for (x1, y1, x2, y2, cls, conf) in kept:
    color = colors.get(cls, (0,255,0))
    label = f"{cls} {conf*100:.0f}%" if SHOW_CONF else cls

    # Rectángulo más fino
    cv2.rectangle(draw, (x1, y1), (x2, y2), color, RECT_THICKNESS)

    # Texto más chico, fondo proporcional
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, FONT_THICKNESS)
    cv2.rectangle(draw, (x1, max(0, y1 - th - 4)), (x1 + tw + 4, y1), color, -1)
    cv2.putText(draw, label, (x1 + 2, y1 - 3),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0,0,0),
                FONT_THICKNESS, cv2.LINE_AA)

# === Mostrar y guardar ===
side = cv2.hconcat([img, draw])
os.makedirs("outputs2", exist_ok=True)
base = osp.splitext(osp.basename(IMG_PATH))[0]
out2 = f"outputs2/{base}_side_by_side_m2.jpg"

cv2.imwrite(out2, side)

cv2.imshow("Detecciones", draw)

# Espera hasta 10 segundos (10000 ms) o hasta que se presione una tecla
key = cv2.waitKey(10000)  # espera 10 segundos

# Si no se presiona nada (key == -1), cierra igual
cv2.destroyAllWindows()

print("Guardado:", out2)

Guardado: outputs2/prueba10_side_by_side_m2.jpg


In [84]:
## MODELO 3


from inference_sdk import InferenceHTTPClient
import cv2
import os, os.path as osp

# === Configuración ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"       # tu API key de Roboflow
MODEL_ID = "football-i1lvk/1"           # modelo a usar
IMG_PATH = "image/prueba1.jpg"          # imagen de entrada

# === Cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === Carga imagen y upscale suave ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"

H, W = orig.shape[:2]
target_long = 1280  # subir tamaño ayuda a ver objetos chicos
scale = max(1.0, target_long / max(H, W))
newW, newH = int(W * scale), int(H * scale)
img = cv2.resize(orig, (newW, newH), interpolation=cv2.INTER_CUBIC)
draw = img.copy()


# === Inferencia ===
res = client.infer(img, model_id=MODEL_ID)  # umbral 0.30
preds = res.get("predictions", [])

# === Filtros suaves ===
USE_PITCH_SOFT = True
USE_AR_FILTER  = False

if USE_PITCH_SOFT:
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def green_ratio(x1, y1, x2, y2):
    if not USE_PITCH_SOFT: return 1.0
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(newW-1, x2); y2 = min(newH-1, y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return 0.0
    return (crop > 0).mean()

def plausible_person_box(x1, y1, x2, y2):
    if not USE_AR_FILTER: return True
    w = max(1, x2 - x1); h = max(1, y2 - y1)
    ar = h / w
    return 0.9 <= ar <= 7.0

TH = {"player":0.35, "goalie":0.35, "referee":0.35, "ball":0.25}
colors = {"player":(0,255,0), "goalie":(255,0,255), "referee":(0,165,255), "ball":(0,255,255)}

kept = []
for p in preds:
    cls  = p["class"]
    conf = float(p["confidence"])
    if conf < TH.get(cls, 0.35): 
        continue

    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2):
            continue
        gr = green_ratio(x1, y1, x2, y2)
        if gr < 0.10:
            conf *= 0.6
            if conf < TH.get(cls, 0.35): 
                continue

    kept.append((x1, y1, x2, y2, cls, conf))

# === Dibujar con etiquetas más pequeñas y finas ===
FONT_SCALE = 0.35          # antes 0.6
FONT_THICKNESS = 1        # antes 2
RECT_THICKNESS = 1        # rectángulo más fino
SHOW_CONF = True          # ponelo en False si querés ocultar el porcentaje

for (x1, y1, x2, y2, cls, conf) in kept:
    # --- limpiar etiquetas largas automáticas ---
    if "roboflow.com" in cls.lower() or "exported" in cls.lower():
        cls = "player"
    if len(cls) > 25:  # por si hay otros textos largos
        cls = "player"
    
    color = colors.get(cls, (0,255,0))
    label = f"{cls} {conf*100:.0f}%" if SHOW_CONF else cls

    # Rectángulo más fino
    cv2.rectangle(draw, (x1, y1), (x2, y2), color, RECT_THICKNESS)

    # Texto más chico, fondo proporcional
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, FONT_THICKNESS)
    cv2.rectangle(draw, (x1, max(0, y1 - th - 4)), (x1 + tw + 4, y1), color, -1)
    cv2.putText(draw, label, (x1 + 2, y1 - 3),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0,0,0),
                FONT_THICKNESS, cv2.LINE_AA)

# === Mostrar y guardar ===
side = cv2.hconcat([img, draw])
os.makedirs("outputs3", exist_ok=True)
base = osp.splitext(osp.basename(IMG_PATH))[0]
out2 = f"outputs3/{base}_side_by_side_m3.jpg"

cv2.imwrite(out2, side)

cv2.imshow("Detecciones", draw)

# Espera hasta 10 segundos (10000 ms) o hasta que se presione una tecla
key = cv2.waitKey(10000)  # espera 10 segundos

# Si no se presiona nada (key == -1), cierra igual
cv2.destroyAllWindows()

print("Guardado:", out2)

Guardado: outputs3/prueba1_side_by_side_m3.jpg


In [74]:
## MODELO 4

from inference_sdk import InferenceHTTPClient
import cv2
import os, os.path as osp

# === Configuración ===
API_KEY  = "SfEb7vZiIzSWMUlRWTWM"        # tu API Key de Roboflow
MODEL_ID = "football-4sbwb/1"            # ID del nuevo modelo
IMG_PATH = "image/prueba10.jpg"           # ruta a la imagen que quieras probar

# === Cliente Roboflow ===
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# === Carga imagen y upscale suave ===
orig = cv2.imread(IMG_PATH)
assert orig is not None, f"No se pudo leer {IMG_PATH}"

H, W = orig.shape[:2]
target_long = 1280  # subir tamaño ayuda a ver objetos chicos
scale = max(1.0, target_long / max(H, W))
newW, newH = int(W * scale), int(H * scale)
img = cv2.resize(orig, (newW, newH), interpolation=cv2.INTER_CUBIC)
draw = img.copy()


# === Inferencia ===
res = client.infer(img, model_id=MODEL_ID)  # umbral 0.30
preds = res.get("predictions", [])

# === Filtros suaves ===
USE_PITCH_SOFT = True
USE_AR_FILTER  = False

if USE_PITCH_SOFT:
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv, (30, 30, 30), (90, 255, 255))

def green_ratio(x1, y1, x2, y2):
    if not USE_PITCH_SOFT: return 1.0
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(newW-1, x2); y2 = min(newH-1, y2)
    crop = green_mask[y1:y2, x1:x2]
    if crop.size == 0: return 0.0
    return (crop > 0).mean()

def plausible_person_box(x1, y1, x2, y2):
    if not USE_AR_FILTER: return True
    w = max(1, x2 - x1); h = max(1, y2 - y1)
    ar = h / w
    return 0.9 <= ar <= 7.0

TH = {"player":0.35, "goalie":0.35, "referee":0.35, "ball":0.25}
colors = {"player":(0,255,0), "goalie":(255,0,255), "referee":(0,165,255), "ball":(0,255,255)}

kept = []
for p in preds:
    cls  = p["class"]
    conf = float(p["confidence"])
    if conf < TH.get(cls, 0.35): 
        continue

    x, y, w, h = p["x"], p["y"], p["width"], p["height"]
    x1, y1 = int(x - w/2), int(y - h/2)
    x2, y2 = int(x + w/2), int(y + h/2)

    if cls != "ball":
        if not plausible_person_box(x1, y1, x2, y2):
            continue
        gr = green_ratio(x1, y1, x2, y2)
        if gr < 0.10:
            conf *= 0.6
            if conf < TH.get(cls, 0.35): 
                continue

    kept.append((x1, y1, x2, y2, cls, conf))

# === Dibujar con etiquetas más pequeñas y finas ===
FONT_SCALE = 0.35          # antes 0.6
FONT_THICKNESS = 1        # antes 2
RECT_THICKNESS = 1        # rectángulo más fino
SHOW_CONF = True          # ponelo en False si querés ocultar el porcentaje

for (x1, y1, x2, y2, cls, conf) in kept:
    color = colors.get(cls, (0,255,0))
    label = f"{cls} {conf*100:.0f}%" if SHOW_CONF else cls

    # Rectángulo más fino
    cv2.rectangle(draw, (x1, y1), (x2, y2), color, RECT_THICKNESS)

    # Texto más chico, fondo proporcional
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, FONT_THICKNESS)
    cv2.rectangle(draw, (x1, max(0, y1 - th - 4)), (x1 + tw + 4, y1), color, -1)
    cv2.putText(draw, label, (x1 + 2, y1 - 3),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0,0,0),
                FONT_THICKNESS, cv2.LINE_AA)

# === Mostrar y guardar ===
side = cv2.hconcat([img, draw])
os.makedirs("outputs4", exist_ok=True)
base = osp.splitext(osp.basename(IMG_PATH))[0]
out2 = f"outputs4/{base}_side_by_side_m4.jpg"

cv2.imwrite(out2, side)

cv2.imshow("Detecciones", draw)

# Espera hasta 10 segundos (10000 ms) o hasta que se presione una tecla
key = cv2.waitKey(10000)  # espera 10 segundos

# Si no se presiona nada (key == -1), cierra igual
cv2.destroyAllWindows()

print("Guardado:", out2)


Guardado: outputs4/prueba10_side_by_side_m4.jpg
